In [ ]:
import sys, os

# Add the project root to the Python path
project_root = os.path.abspath("../..")
sys.path.append(project_root)

from utils.experiment_tracker import ExperimentTracker
from utils.constants import METRICS_DB_PATH, CLEANED_DIR
from utils.model_evaluator import ModelEvaluator

tracker = ExperimentTracker()
df_all_evals = tracker.get_all_experiment_df()



evaluator = ModelEvaluator()
df_enriched = evaluator.enrich_model_summary(df_all_evals)
display(df_enriched)  # ou print(df_enriched) si hors notebook


e:\_SoftEng\_BeCode\real-estate-price-predictor\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,id,timestamp,model,dataset,experiment,cleaning_version_id,mae,rmse,r2,type,rank_r2,rmse/mae,best
0,25,2025-06-28 23:24:12,Linear Regression (All Features),immoweb_real_estate_cleaned_for_ml_20250628_23...,Linear Regression (All Features),202506282323,76 898.76 €,109 935.46 €,0.7230,Linear,7,1.43,
1,26,2025-06-28 23:24:41,Random Forest (All Features),immoweb_real_estate_cleaned_for_ml_20250628_23...,Random Forest (All Features),202506282323,68 324.02 €,101 428.25 €,0.7642,Tree,5,1.48,
2,27,2025-06-28 23:24:50,Random Forest (Top 10 Features),immoweb_real_estate_cleaned_for_ml_20250628_23...,Random Forest (Top 10 Features),202506282323,71 734.61 €,106 140.03 €,0.7418,Tree,6,1.48,
3,28,2025-06-28 23:25:34,XGBoost + Optuna CV (All Features),immoweb_real_estate_cleaned_for_ml_20250628_23...,XGBoost with Optuna (All Features),202506282323,57 331.57 €,82 338.95 €,0.8447,Boosting,4,1.44,
4,29,2025-06-28 23:25:34,XGBoost + Optuna CV (Top RF Features),immoweb_real_estate_cleaned_for_ml_20250628_23...,XGBoost with Optuna (Top RF Features),202506282323,48 709.38 €,69 322.48 €,0.8899,Boosting,3,1.42,
5,30,2025-06-28 23:29:11,CatBoost + Optuna CV (All Features),immoweb_real_estate_cleaned_for_ml_20250628_23...,CatBoost with Optuna (All Features),202506282323,41 446.73 €,57 924.16 €,0.9231,Boosting,2,1.40,
6,31,2025-06-28 23:29:11,CatBoost + Optuna CV (Top RF Features),immoweb_real_estate_cleaned_for_ml_20250628_23...,CatBoost with Optuna (Top RF Features),202506282323,39 778.00 €,55 472.29 €,0.9295,Boosting,1,1.39,✓
